In [ ]:
%load_ext autoreload
%autoreload 2
import pycolmap
import pyceres
import numpy as np
from hloc.utils import viz_3d
from copy import deepcopy

def look_at(at, w_p_c, up=(0, 0, 1)):
    up = np.asarray(up)
    normalize = lambda x: x / np.linalg.norm(x).clip(min=1e-5)
    z_axis = normalize(at - w_p_c)
    x_axis = normalize(np.cross(z_axis, up))
    y_axis = normalize(np.cross(z_axis, x_axis))
    w_r_c = np.stack([x_axis, y_axis, z_axis], axis=-1)
    return pycolmap.Rigid3d(pycolmap.Rotation3d(w_r_c), w_p_c)

def randn_with_norm(norm: float, *args, **kwargs):
    x = np.random.randn(*args, **kwargs)
    x *= norm / np.linalg.norm(x, axis=-1, keepdims=True)
    return x

def sample_pose_perturbation(max_dr: float, max_dt: float):
    aa = randn_with_norm(np.deg2rad(max_dr), 3)
    t = randn_with_norm(max_dt, 3)
    return pycolmap.Rigid3d(pycolmap.Rotation3d(aa), t)

def get_gt_depth(world_from_cam, camera, xy):
    # Assume all points lie on the X-Z plane.
    uv = camera.cam_from_img(xy)
    R  = world_from_cam.rotation.matrix()
    depth = -((uv @ R[1, :2]) + world_from_cam.translation[1]) / R[1, 2]
    assert (depth > 0).all()
    return depth

def sample_depth_noise(d, stddev=0.1, min=-0.5, max=0.5):
    return 2 ** ((np.random.randn(*d.shape) * stddev).clip(min, max))

def apply_shift_scale(d, shift_scale):
    return shift_scale[0] + d * shift_scale[1]

def revert_shift_scale(d, shift_scale):
    return (d - shift_scale[0]) / shift_scale[1]

np.random.seed(3)
center = np.zeros(3)
p_min = np.array([-3, -5, -3])
p_max = np.array([3, -4, 3])
n_views = 2
world_from_gt = [look_at(center, np.random.rand(3) * (p_max - p_min) + p_min) for _ in range(n_views)]
gt_from_world = [w_t_gt.inverse() for w_t_gt in world_from_gt]
world_from_init = [w_t_gt * sample_pose_perturbation(2, 0.1) for w_t_gt in world_from_gt]
init_from_world = [w_t_i.inverse() for w_t_i in world_from_init]
camera = pycolmap.Camera.create(1, 'SIMPLE_PINHOLE', 320, 640, 480)

n_points = 200
image_size = np.array([camera.width, camera.height])
p2d_0 = np.random.rand(n_points, 2) * image_size
depth_gt_0 = get_gt_depth(world_from_gt[0], camera, p2d_0)
p3d_gt = world_from_gt[0] * (np.concatenate([camera.cam_from_img(p2d_0), np.ones((n_points, 1))], -1) * depth_gt_0[:, None])

p2d_1 = camera.img_from_cam(world_from_gt[1].inverse() * p3d_gt)
mask = np.all((p2d_1>=0) & (p2d_1 <= image_size), -1)
print(mask.sum())
p3d_gt = p3d_gt[mask]
p2d_gt = [p2d_0[mask], p2d_1[mask]]
depths_gt = [
    depth_gt_0[mask],
    (world_from_gt[1].inverse() * p3d_gt)[:, -1]
]

p3d_noisy = p3d_gt + np.random.randn(len(p3d_gt), 3)*0.1
shift_scale_gt = [
    np.array([0., 1.]),
    # np.array([0., 1.]),
    np.array([0., 2.]),
]
shift_scale_init = [np.array([0., 1.]) for _ in range(2)]
depths_obs = [
    revert_shift_scale(d * sample_depth_noise(d, stddev=0.001), ss) for d, ss in zip(depths_gt, shift_scale_gt)
]

In [ ]:
fig = viz_3d.init_figure()
viz_3d.plot_points(fig, p3d_gt, color='red', ps=3)
for i, w_t_c in enumerate(world_from_gt):
    viz_3d.plot_camera(
        fig, w_t_c.rotation.matrix(), w_t_c.translation, camera.calibration_matrix(),
        size=5, name=str(i), text='', color='lime')
fig.show()

In [ ]:
def solve(prob):
    print(
        prob.num_parameter_blocks(),
        prob.num_parameters(),
        prob.num_residual_blocks(),
        prob.num_residuals(),
    )
    options = pyceres.SolverOptions()
    options.linear_solver_type = pyceres.LinearSolverType.DENSE_QR
    options.minimizer_progress_to_stdout = True
    options.num_threads = -1
    summary = pyceres.SolverSummary()
    pyceres.solve(options, prob, summary)
    print(summary.BriefReport())

p3d_opt = deepcopy(p3d_noisy)
shift_scale_opt = deepcopy(shift_scale_gt)

prob = pyceres.Problem()
loss = pyceres.TrivialLoss()
for i in range(n_views):
    for j in range(len(p3d_opt)):
        cost = pycolmap.cost_functions.ScaledDepthErrorCost(gt_from_world[i], depths_obs[i][j])
        prob.add_residual_block(cost, loss, [p3d_opt[j], shift_scale_opt[i]])
    prob.set_parameter_block_constant(shift_scale_opt[i])
# prob.set_parameter_block_constant(shift_scale_opt[0])
# prob.set_manifold(shift_scale_opt[1], pyceres.SubsetManifold(2, [0]))
solve(prob)

In [ ]:
[np.abs(gt_t_w.rotation * (p3d_noisy - p3d_gt))[:,-1].mean() for gt_t_w in gt_from_world]

In [ ]:
[np.abs(gt_t_w.rotation * (p3d_opt - p3d_gt))[:,-1].mean() for gt_t_w in gt_from_world]

In [ ]:
shift_scale_opt, shift_scale_init, shift_scale_gt